<a href="https://colab.research.google.com/github/aliash98/Search2Vec-BScThesis/blob/main/Implementation%20on%20AOL%20Dataset/AOL_User_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from datetime import datetime
import pandas as pd
import numpy as np
import re
import json
import warnings
import gc 
from tqdm import tqdm, trange
from scipy import spatial
import math

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
data = pd.read_csv (r'./gdrive/MyDrive/Colab Notebooks/AOL_Implementation/train.csv')

In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 567047 entries, 0 to 567046
Data columns (total 5 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   AnonID     567047 non-null  int64  
 1   Query      567047 non-null  object 
 2   QueryTime  567047 non-null  object 
 3   ItemRank   300763 non-null  float64
 4   ClickURL   300763 non-null  object 
dtypes: float64(1), int64(1), object(3)
memory usage: 21.6+ MB


In [ ]:
data['AnonID'].value_counts().head(5)

2426641    1487
700766     1104
2067984    1096
3033935    1045
3318459    1026
Name: AnonID, dtype: int64

In [ ]:
data['AnonID'].value_counts().describe()
# data['AnonID'].value_counts().count()
# Useful information about the users
data['AnonID'].value_counts().head()

2426641    1487
700766     1104
2067984    1096
3033935    1045
3318459    1026
Name: AnonID, dtype: int64

In [ ]:
# Discounting users with less than 5 queries
print((data['AnonID'].value_counts()==5).argmax())


data = data[data.AnonID.isin(data['AnonID'].value_counts().head(18781).index)]
# data.info()

18781


In [ ]:
# We should construct an 45004 * 45004 
user_num = data['AnonID'].value_counts().count()
W = np.zeros((user_num, user_num), dtype=float)
# user_num

In [ ]:
user_df = data['AnonID'].value_counts()
user_df.describe()
user_df.head()

2426641    1487
700766     1104
2067984    1096
3033935    1045
3318459    1026
Name: AnonID, dtype: int64

In [ ]:
counter = 0
index_dict = {}
for index, value in user_df.items():
  if counter < 5:
    print(f"Index : {index}, Value : {value}")
  index_dict[index] = counter
  counter += 1
len(index_dict)

Index : 2426641, Value : 1487
Index : 700766, Value : 1104
Index : 2067984, Value : 1096
Index : 3033935, Value : 1045
Index : 3318459, Value : 1026


18781

In [ ]:
# construction of the numbers a docmunet has been clicked
urls_df = data['ClickURL'].value_counts()
# if it's slow then we can put it into a dict

In [ ]:
# Checking the urls_df
urls_df.loc['http://www.google.com']

23502

In [ ]:
urls_df.head(5)
data.dropna(subset = ["ClickURL"], inplace=True)

In [ ]:
# Preprocess, making dictionaries for clicked urls
user_url_clicked_dict = {}
for index_i, value_i in user_df.items():
  # user_df.index[0]
  data_i = data[data['AnonID'] == index_i]
  a = set(data_i['ClickURL'])
  user_url_clicked_dict[index_i] = a
# data_i = data_i.drop_duplicates(subset=['ClickURL'])
# print(a)
print(len(user_url_clicked_dict))

18781


In [ ]:
user_url_clicked_dict[user_df.index[1879]]
user_df.loc[2426641]

1487

In [ ]:
# Continue of preprocess
# Forming a dict for finding the number of documents each user has clicked
num_urls_for_each_user = {}
for index_i, value_i in user_df.items():
  num_urls_for_each_user[index_i] = value_i
num_urls_for_each_user[700766]
# Forming a dict for finding the number of clicks each document has
num_clicks_for_each_url = {}
for index_i, value_i in urls_df.items():
  num_clicks_for_each_url[index_i] = value_i
urls_df.head()
num_clicks_for_each_url['http://www.mapquest.com']

5843

In [ ]:
x_temp = 0
for user in user_url_clicked_dict.keys():
  for user_prime in user_url_clicked_dict.keys():
    if user == user_prime:
      continue
    intersection = user_url_clicked_dict[user].intersection(user_url_clicked_dict[user_prime])
    sigma = 0
    for item in intersection:
      sigma += 1 / np.log(1 + num_clicks_for_each_url[item])
    W[index_dict[user], index_dict[user_prime]] = sigma / math.sqrt(num_urls_for_each_user[user] * num_urls_for_each_user[user_prime])
  print(index_dict[user])  




# x_temp = 0
# for index_i, value_i in user_df.items():
#   x_temp += 1
#   data_i = data[data['AnonID'] == index_i]
#   data_i = data_i.drop_duplicates(subset=['ClickURL'])
#   for index_j, value_j in user_df.items():
#     x_temp += 1
#     if index_i == index_j:
#       continue
#     # Obtaining clicked documents of users
#     data_j = data[data['AnonID'] == index_j]
#     data_j = data_j.drop_duplicates(subset=['ClickURL'])
#     # intersection = pd.merge(data_i, data_j, how='inner', on=['ClickURL'])
#     intersection = pd.merge(data_i, data_j, how='inner', on=["ClickURL"])
#     # print(len(intersection))
#     # print(len(data_i))
#     # print(len(data_j))
#     # print(data_i)
#     # print(data_j)
#     # print(intersection.iloc[0])
#     sigma = 0
#     for index_k, value_k in intersection['ClickURL'].items():
#       sigma += 1 / np.log(1 + urls_df.loc[value_k])
#     W[index_dict[index_i], index_dict[index_j]] = sigma / math.sqrt(value_i * value_j)
#   print(index_dict[index_i])    


In [ ]:
pd.DataFrame(W).to_csv(r'./gdrive/MyDrive/Colab Notebooks/AOL_Implementation/W.tsv', sep="\t", index=False, header=None)